In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from statsmodels.regression.dimred import CORE

Simulate data, replace this with the actual data.

In [ ]:
n_person = 27
n_day = np.random.poisson(100, n_person)
p = 10
n = np.sum(n_day)
person = np.concatenate([i*np.ones(k) for i,k in enumerate(n_day)]).astype(int)
day = np.concatenate([np.arange(k) for k in n_day]).astype(int)
C1r = np.random.normal(size=(p, p))
C1 = np.dot(C1r.T, C1r)
C2r = np.random.normal(size=(p, p))
C2 = np.dot(C2r.T, C2r)
person_effects = np.dot(np.random.normal(size=(n_person, p)), C1r)
person_effects = person_effects[person, :]
day_effects = np.dot(np.random.normal(size=(n, p)), C2r)
sleep = person_effects + day_effects

# MANOVA

In [ ]:
def manova1(X, grp, center=True):
    """
    Run MANOVA on the data in X.
    """
    if center:
        X = X - X.mean(0)
    dg = np.diff(grp)
    assert np.all(dg >= 0) # grp must be sorted
    ii = np.flatnonzero(dg > 0) + 1
    x = np.split(X, ii)
    M = np.stack([z.mean(0) for z in x])
    w = np.asarray([z.shape[0] for z in x])
    Mw = np.sqrt(w)[:, None] * M
    H = np.dot(Mw.T, Mw)
    E = np.dot(X.T, X) - H
    R = np.linalg.solve(E, H)
    ee, _ = np.linalg.eig(R)
    ee = np.sort(ee)[::-1]
    pillai = np.sum(ee / (1 + ee))
    lawley = np.sum(ee)
    wilks = np.prod(1 / (1 + ee))
    roy = ee[0] / (1 + ee[0])
    return H, E, ee, (pillai, lawley, wilks, roy)

In [ ]:
def manova(X, grp, nrep=1000):
    """
    Run MANOVA on the data in X, using randomization to calibrate the statistics.
    """
    n = X.shape[0]
    X0 = X - X.mean(0)
    H, E, ee, stats = manova1(X0, grp)
    stats0 = np.empty((nrep, 4))
    for i in range(nrep):
        ii = np.random.permutation(n)
        _, _, ee0, s = manova1(X[ii, :], grp, center=False)
        stats0[i, :] = s
    return H, E, ee, stats, stats0

Transform the MANOVA summary statistics to make them more interpretable.

In [ ]:
def xstat_manova(stats, stats0, p):
    stats = np.copy(stats)
    stats0 = np.copy(stats0)
    # Pillai becomes the mean PVE
    stats[0] /= p
    stats0[:, 0] /= p
    # Lawley becomes the mean SNR
    stats[1] /= p
    stats0[:, 1] /= p
    # Wilks becomes the geometric mean of 1 - PVE
    stats[2] = np.power(stats[2], 1/p)
    stats0[:, 2] = np.power(stats0[:, 2], 1/p)
    # Not sure what to do with this...
    stats[3] = np.power(stats[3], 1/p)
    stats0[:, 3] = np.power(stats0[:, 3], 1/p)
    return stats, stats0

In [ ]:
H, E, ee, stats, stats0 = manova(sleep, person)

statsx, stats0x = xstat_manova(stats, stats0, p)

print(ee)
print(statsx)

for (k, ti) in enumerate(["Pillai", "Lawley", "Wilks", "Roy"]):
    plt.hist(stats0x[:, k], bins=30, color="lightblue")
    plt.axvline(statsx[k], color="red")
    plt.title(ti)
    plt.show()

# PCA/Biplots

In [ ]:
def biplot(X, j=0, k=1):
    X = np.copy(X)
    X -= X.mean()
    X -= X.mean(0)
    X -= X.mean(1)[:, None]
    u, s, vt = np.linalg.svd(X, 0)
    v = vt.T
    u *= (s**0.5)
    v *= (s**0.5)
    f = np.sqrt(X.shape[0] / X.shape[1]) # May need to adjust this
    v /= f
    plt.clf()
    plt.plot(u[:, j], u[:, k], "o", color="black", alpha=0.1)
    for i in range(v.shape[0]):
        plt.annotate("v%d" % i, xy=(0, 0), xytext=(v[i, j], v[i, k]), 
                     arrowprops=dict(facecolor='black', arrowstyle="<-"))
    plt.xlabel("Component %d" % j)
    plt.ylabel("Component %d" % k)
        
biplot(sleep)

# CORE

In [ ]:
tab20 = matplotlib.cm.get_cmap("tab20").colors

In [ ]:
m = CORE(person, sleep, 2)
r = m.fit(maxiter=1000)
qq = r.params
cc = [np.dot(qq.T, np.dot(c, qq)) for c in m.covs]
ci = [np.linalg.eigh(c) for c in cc]

plt.clf()
plt.grid(True)
plt.xlim(-3, 3)
plt.ylim(-3, 3)
for j,(a, b) in enumerate(ci):
    plt.plot(a[0]*np.r_[-b[0, 0], b[0, 0]], a[0]*np.r_[-b[1, 0], b[1, 0]], ":", color=tab20[j % 20])
    plt.plot(a[1]*np.r_[-b[0, 1], b[0, 1]], a[1]*np.r_[-b[1, 1], b[1, 1]], "-", color=tab20[j % 20])